In [3]:
import pandas as pd
from py2neo import Graph, Node, Relationship

class Neo4jImporter:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def clear_database(self):
        self.graph.delete_all()

    def import_csv(self):
        # 导入Paragraphs，避免重复
        paragraphs = pd.read_csv('/Users/qinmingming/workspace/rag_neo4j_llm_datacross/neo4j/neo4j_csv/Paragraphs.csv')
        for index, row in paragraphs.iterrows():
            node = Node('Paragraph', id=row['ParagraphID'], text=row['Paragraph'], name=row['name'])
            self.graph.merge(node, 'Paragraph', 'id')
        print("import paragraph")

        # 导入Keywords，避免重复
        keywords = pd.read_csv('/Users/qinmingming/workspace/rag_neo4j_llm_datacross/neo4j/neo4j_csv/Keywords.csv')
        for index, row in keywords.iterrows():
            node = Node('Keyword', id=row['KeywordID'], keyword=row['Keywords'], name=row['name'])
            self.graph.merge(node, 'Keyword', 'id')
        print("import keywords")

        # 导入Category，避免重复
        categories = pd.read_csv('/Users/qinmingming/workspace/rag_neo4j_llm_datacross/neo4j/neo4j_csv/Categories.csv')
        for index, row in categories.iterrows():
            node = Node('Category', id=row['CategoryID'], category=row['Category'], name=row['name'])
            self.graph.merge(node, 'Category', 'id')
        print("import categories")

        # 导入Country，避免重复
        countries = pd.read_csv('/Users/qinmingming/workspace/rag_neo4j_llm_datacross/neo4j/neo4j_csv/Countries.csv')
        for index, row in countries.iterrows():
            node = Node('Country', id=row['CountryID'], country=row['Country'], name=row['name'])
            self.graph.merge(node, 'Country', 'id')
        print("import countries")

        # 创建Keyword到Category的关系，避免重复
        keyword_categories = pd.read_csv('/Users/qinmingming/workspace/rag_neo4j_llm_datacross/neo4j/neo4j_csv/KeywordCategories.csv')
        for index, row in keyword_categories.iterrows():
            keyword_node = self.graph.nodes.match('Keyword', id=row['KeywordID']).first()
            category_node = self.graph.nodes.match('Category', id=row['CategoryID']).first()
            if keyword_node and category_node:
                rel = Relationship(keyword_node, 'BELONGS_TO', category_node)
                self.graph.merge(rel)
        print("keyword_categories")

        # 导入ParagraphKeywords关系
        paragraph_keywords = pd.read_csv('/Users/qinmingming/workspace/rag_neo4j_llm_datacross/neo4j/neo4j_csv/ParagraphKeywords.csv')
        for index, row in paragraph_keywords.iterrows():
            paragraph_node = self.graph.nodes.match('Paragraph', id=row['ParagraphID']).first()
            keyword_node = self.graph.nodes.match('Keyword', id=row['KeywordID']).first()
            if paragraph_node and keyword_node:
                rel = Relationship(paragraph_node, 'CONTAINS', keyword_node)
                self.graph.merge(rel)
        print("paragraph_keywords")

        # 导入CategoryCountries关系
        category_countries = pd.read_csv('/Users/qinmingming/workspace/rag_neo4j_llm_datacross/neo4j/neo4j_csv/CategoryCountries.csv')
        for index, row in category_countries.iterrows():
            category_node = self.graph.nodes.match('Category', id=row['CategoryID']).first()
            country_node = self.graph.nodes.match('Country', id=row['CountryID']).first()
            if category_node and country_node:
                rel = Relationship(category_node, 'CONTAINS', country_node)
                self.graph.merge(rel)
        print("category_countries")

if __name__ == "__main__":
    uri = "bolt://localhost:7687"  
    user = "neo4j"  
    password = "neo4j123"  
    importer = Neo4jImporter(uri, user, password)
    importer.clear_database()  # 清空数据库
    importer.import_csv()  # 导入CSV数据


KeyboardInterrupt: 

In [4]:
uri = "bolt://localhost:7687"  
user = "neo4j"  
password = "neo4j123"  
graph = Graph(uri, auth=(user, password))

In [5]:
graph.delete_all()

In [6]:
keywords = pd.read_csv('/Users/qinmingming/workspace/rag_neo4j_llm_datacross/neo4j/neo4j_csv/Keywords.csv')
for index, row in keywords.iterrows():
    node = Node('Keyword', id=row['KeywordID'], keyword=row['Keywords'], name=row['name'])
    graph.merge(node, 'Keyword', 'id')
print("import keywords")

import keywords


In [1]:
# !pip3 install neo4j

In [10]:
from neo4j import GraphDatabase

class Neo4jImporter:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def clear_database(self):
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")

    def import_csv(self):
        with self.driver.session() as session:
            # 导入Paragraphs，避免重复
            session.run("""
            LOAD CSV WITH HEADERS FROM 'file:///Paragraphs.csv' AS row
            MERGE (p:Paragraph {id: row.ParagraphID})
            ON CREATE SET p.text = row.Paragraph, p.name = row.name
            """)

            # 导入Keywords，避免重复
            session.run("""
            LOAD CSV WITH HEADERS FROM 'file:///Keywords.csv' AS row
            MERGE (k:Keyword {id: row.KeywordID})
            ON CREATE SET k.keyword = row.Keywords, k.name = row.name
            """)

            # 导入Category，避免重复
            session.run("""
            LOAD CSV WITH HEADERS FROM 'file:///Categories.csv' AS row
            MERGE (c:Category {id: row.CategoryID})
            ON CREATE SET c.category = row.Category, c.name = row.name
            """)

            # 导入Country，避免重复
            session.run("""
            LOAD CSV WITH HEADERS FROM 'file:///Countries.csv' AS row
            MERGE (c:Country {id: row.CountryID})
            ON CREATE SET c.country = row.Country, c.name = row.name
            """)

            # 创建Keyword到Category的关系，避免重复
            session.run("""
            LOAD CSV WITH HEADERS FROM 'file:///KeywordCategories.csv' AS row
            MATCH (k:Keyword {id: row.KeywordID}), (c:Category {id: row.CategoryID})
            MERGE (k)-[:BELONGS_TO]->(c)
            """)
            # 导入ParagraphKeywords关系
            session.run("""
            LOAD CSV WITH HEADERS FROM 'file:///ParagraphKeywords.csv' AS row
            MATCH (paragraph:Paragraph {id: row.ParagraphID}), (keyword:Keyword {id: row.KeywordID})
            MERGE (paragraph)-[:CONTAINS]->(keyword)
            """)

             # 导入CategoryCountries关系
            session.run("""
            LOAD CSV WITH HEADERS FROM 'file:///CategoryCountries.csv' AS row
            MATCH (category:Category {id: row.CategoryID}), (country:Country {id: row.CountryID})
            MERGE (category)-[:CONTAINSCOUNTRY]->(country)
            """)



if __name__ == "__main__":
    uri = "bolt://localhost:7687"  
    user = "neo4j"  
    password = "neo4j123"  #
    importer = Neo4jImporter(uri, user, password)
    importer.clear_database()  # 清空数据库
    importer.import_csv()  # 导入CSV数据
    importer.close()


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.CartesianProductWarning} {category: } {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by using OPTIONAL MATCH (identifier is: (c))} {position: line: 3, column: 25, offset: 91} for query: "\n            LOAD CSV WITH HEADERS FROM 'file:///KeywordCategories.csv' AS row\n            MATCH (k:Keyword {id: row.KeywordID}), (c:Category {id: row.CategoryID})\n            MERGE (k)-[:BELONGS_TO]->(c)\n            "
Received notification from DBMS server: {severity: WARNING} {cod

In [11]:
uri = "bolt://localhost:7687"  
user = "neo4j"  
password = "neo4j123"  #

driver = GraphDatabase.driver(uri, auth=(user, password))
paragraph_list = []
def search_paragraphs_by_category(category_name):
    
    query = """
    MATCH (c:Category {category: $category_name})<-[:BELONGS_TO]-(k:Keyword)<-[:CONTAINS]-(p:Paragraph)
    RETURN p.text AS paragraph_text
    """
    with driver.session() as session:
        result = session.run(query, category_name=category_name)
        for record in result:
            paragraph_list.append(record)
            # paragraph_list.append(record["text"])
    return paragraph_list



In [15]:
category_name = "隐私委员会"

paragraph_list = search_paragraphs_by_category(category_name)

In [16]:
len(paragraph_list)

1

In [20]:
paragraph_list[0]['paragraph_text']

'菲律宾数据隐私法政策声明第二章国家隐私委员会 菲律宾数据隐私法政策声明7。国家隐私委员会的职能。为了管理和实施本法的规 定，为了监督和确保国家遵守为数据保护制定的国际标准，特此设立一个独立的机 构，称为国家隐私委员会，其职能如下: 菲律宾数据隐私法政策声明(a)确保个人信息控制者遵守本法的规定; 菲律宾数据隐私法政策声明(b)接收投诉，开展调查，通过使用替代性争议解决程序促 进或促成投诉的解决，对影响任何个人信息的事项进行裁决、裁决、赔偿，就其发起 的任何调查的处理和解决准备报告，并在其认为适当的情况下公布任何该等报告;但在 解决任何投诉或调查时(除非双方达成友好解决)，委员会应作为合议制机构。为此目 的，可以允许委员会访问任何投诉所涉及的个人信息，并收集履行本法规定的职能所 必需的信息; 菲律宾数据隐私法政策声明(c)在发现个人信息处理将损害国家安全和公共利益时，发 布停止令，对个人信息处理实施临时或永久禁令; 菲律宾数据隐私法政策声明(d)强迫或请愿任何实体、政府机构或工具遵守其命令或就 影响数据隐私的事项采取行动; 菲律宾数据隐私法政策声明(e)监督其他政府机构或机构对其安全和技术措施的合规 性，并建议必要的行动， 以满足本法规定的个人信息保护的最低标准; 菲律宾数据隐私法政策声明(f)与其他政府机构和私营部门协调，努力制定和实施加强 国内个人信息保护的计划和政策; 菲律宾数据隐私法政策声明(g)定期出版与数据保护有关的所有法律指南; 菲律宾数据隐私法政策声明(h)出版机构记录和通知系统汇编，包括索引和其他查找工 具; 菲律宾数据隐私法政策声明(i)向司法部(DOJ)建议起诉和实施本法第25 至29 节规定的 处罚; 菲律宾数据隐私法政策声明(j)审查、批准、拒绝或要求修改个人信息控制者自愿遵守 的隐私法典:前提是， 隐私法典应遵守本法所体现的基础数据隐私原则:进一步前提 是，该等隐私法典可包括针对任何参与的个人信息控制者的投诉的私人争议解决机 制。为此目的，对于上述监管机构依法被授权主要监管的个人、实体、商业活动和商'

In [2]:
import pandas as pd
dict_path = './neo4j_csv/Keywords.csv'
keyword_pd = pd.read_csv(dict_path)
print(keyword_pd.shape)
keyword_pd.head(2) 

(1024, 3)


,Keywords,name,KeywordID
0,投诉,投诉,1
1,数据安全法,数据安全法,2


In [4]:
keyword_pd.head(10) 

,Keywords,name,KeywordID
0,投诉,投诉,1
1,数据安全法,数据安全法,2
2,数据安全,数据安全,3
3,数据安全治理,数据安全治理,4
4,网络安全审查办法,网络安全审查办法,5
5,网络数据安全,网络数据安全,6
6,网络安全法,网络安全法,7
7,网络安全审查,网络安全审查,8
8,数据安全监管,数据安全监管,9
9,数据安全行为规范,数据安全行为规范,10


In [5]:
keyword_dict = dict(zip(keyword_pd['Keywords'],keyword_pd['KeywordID']))
keyword_dict

{'投诉': 1,
 '数据安全法': 2,
 '数据安全': 3,
 '数据安全治理': 4,
 '网络安全审查办法': 5,
 '网络数据安全': 6,
 '网络安全法': 7,
 '网络安全审查': 8,
 '数据安全监管': 9,
 '数据安全行为规范': 10,
 '数据安全工作协调机制': 11,
 '军事网络安全保护': 12,
 '数据安全保护义务': 13,
 '数据安全保护': 14,
 '数据开发利用': 15,
 '数据安全标准体系': 16,
 '数据交易管理制度': 17,
 '数据安全风险评估': 18,
 '数据安全管理': 19,
 '数据安全管理制度': 20,
 '数据安全应急处置机制': 21,
 '数据分类分级保护制度': 22,
 '数据安全审查制度': 23,
 '数据安全风险': 24,
 '智能化公共服务': 25,
 '数据依法实施出口管制': 26,
 '网络产品和服务': 27,
 '数据处理服务': 28,
 '数据调取': 29,
 '罚款': 30,
 '数据出境安全评估': 31,
 '数据交易中介服务': 32,
 '政务数据开放平台': 33,
 '数据出境安全评估办法': 34,
 '数据安全保护责任义务': 35,
 '安全评估': 36,
 '数据出境安全管理': 37,
 '网络安全': 38,
 '网络安全战略': 39,
 '网络安全意识': 40,
 '网络治理': 41,
 '个人信息保护法': 42,
 '网络安全保护义务': 43,
 '网络安全行为规范': 44,
 '举报': 45,
 '网络安全服务': 46,
 '网络安全技术产业': 47,
 '网络安全标准体系': 48,
 '网络数据安全保护': 49,
 '网络安全宣传教育': 50,
 '电子身份认证': 51,
 '网络安全等级保护制度': 52,
 '网络可信身份': 53,
 '网络安全认证': 54,
 '网络安全人才培养': 55,
 '网络安全合作': 56,
 '网络安全事件应急预案': 57,
 '网络安全保护职责': 58,
 '网络安全专用产品': 59,
 '网络安全协议': 60,
 '关键信息基础设施保护体系': 61,
 '关键信息基础设施安全规划': 62,
 '网络安全服务机构':

In [7]:
with open("keyword.txt", "w", encoding="utf-8") as f_in:
    for keyword in list(keyword_dict):
        f_in.write(f"{keyword} 100\n")


In [11]:
import jieba 
import pandas as pd
dict_path = './neo4j_csv/Keywords.csv'
keyword_pd = pd.read_csv(dict_path)
# 加载自定义词典
keyword_dict = dict(zip(keyword_pd['Keywords'],keyword_pd['KeywordID']))
jieba.load_userdict("keyword.txt")

# 测试分词
sentence = "请你在java包里面，实现jvm虚拟机的数据安全工作协调机制操作,数据安全管理"
words = jieba.lcut(sentence)
words 

['请',
 '你',
 '在',
 'java',
 '包',
 '里面',
 '，',
 '实现',
 'jvm',
 '虚拟机',
 '的',
 '数据安全工作协调机制',
 '操作',
 ',',
 '数据安全管理']

In [12]:
res = []
for word in words:
    if len(word) > 1 and word in keyword_dict:
        res.append(word)
res 

['数据安全工作协调机制', '数据安全管理']

In [13]:
s = ['请',
 '你',
 '在',
 'java',
 '包',
 '里面',
 '，',
 '实现',
 'jvm',
 '虚拟机',
 '的',
 '数据安全工作协调机制',
 '操作',
 ',',
 '数据安全管理']
print(';'.join(s))

请;你;在;java;包;里面;，;实现;jvm;虚拟机;的;数据安全工作协调机制;操作;,;数据安全管理


In [18]:
from search import search_from_neo4j
user_input = '跟我说一下中国相关的数据安全法以及网络安全监督管理职责'
country = '中国'
res = search_from_neo4j(user_input=user_input,country=country)
res 


{'数据安全法': '中华人民共和国数据安全法第三十六条中华人民共和国主管机关根据有关法律和中华 人民共和国缔结或者参加的国际条约、协定，或者按照平等互惠原则，处理外国司法 或者执法机构关于提供数据的请求。非经中华人民共和国主管机关批准，境内的组 织、个人不得向外国司法或者执法机构提供存储于中华人民共和国境内的数据 中华人民共和国数据安全法第三十七条国家大力推进电子政务建设，提高政务数据的 科学性、准确性、时效性，提升运用数据服务经济社会发展的能力。;中华人民共和国数据安全法第一条为了规范数据处理活动，保障数据安全，促进数据 开发利用，保护个人、组织的合法权益，维护国家主权、安全和发展利益，制定本 法。;中华人民共和国数据安全法第五十条履行数据安全监管职责的国家工作人员玩忽职 守、滥用职权、徇私舞弊的，依法给予处分。;中华人民共和国数据安全法第五十二条违反本法规定，给他人造成损害的，依法承担 民事责任。违反本法规定，构成违反治安管理行为的，依法给予治安管理处罚；构成 犯罪的，依法追究刑事责任。;中华人民共和国数据安全法第七条国家保护个人、组织与数据有关的权益，鼓励数据 依法合理有效利用，保障数据依法有序自由流动，促进以数据为关键要素的数字经济 发展。;中华人民共和国数据安全法第四十六条违反本法第三十一条规定，向境外提供重要数 据的，由有关主管部门责令改正，给予警告，可以并处十万元以上一百万元以下罚 款，对直接负责的主管人员和其他直接责任人员可以处一万元以上十万元以下罚款； 情节严重的，处一百万元以上一千万元以下罚款，并可以责令暂停相关业务、停业整 顿、吊销相关业务许可证或者吊销营业执照，对直接负责的主管人员和其他直接责任 人员处十万元以上一百万元以下罚款。;中华人民共和国数据安全法第三十八条国家机关为履行法定职责的需要收集、使用数 据，应当在其履行法定职责的范围内依照法律、行政法规规定的条件和程序进行；对 在履行职责中知悉的个人隐私、个人信息、商业秘密、保密商务信息等数据应当依法 予以保密，不得泄露或者非法向他人提供。;中华人民共和国数据安全法第十四条国家实施大数据战略，推进数据基础设施建设， 鼓励和支持数据在各行业、各领域的创新应用。省级以上人民政府应当将数字经济发 展纳入本级国民经济和社会发展规划，并根据需要制定数字经济发展规划。;中华人民共和国数据安全法第四十

In [17]:
res


{'数据安全法': '中华人民共和国数据安全法第三十六条中华人民共和国主管机关根据有关法律和中华 人民共和国缔结或者参加的国际条约、协定，或者按照平等互惠原则，处理外国司法 或者执法机构关于提供数据的请求。非经中华人民共和国主管机关批准，境内的组 织、个人不得向外国司法或者执法机构提供存储于中华人民共和国境内的数据 中华人民共和国数据安全法第三十七条国家大力推进电子政务建设，提高政务数据的 科学性、准确性、时效性，提升运用数据服务经济社会发展的能力。;中华人民共和国数据安全法第一条为了规范数据处理活动，保障数据安全，促进数据 开发利用，保护个人、组织的合法权益，维护国家主权、安全和发展利益，制定本 法。;中华人民共和国数据安全法第五十条履行数据安全监管职责的国家工作人员玩忽职 守、滥用职权、徇私舞弊的，依法给予处分。;中华人民共和国数据安全法第五十二条违反本法规定，给他人造成损害的，依法承担 民事责任。违反本法规定，构成违反治安管理行为的，依法给予治安管理处罚；构成 犯罪的，依法追究刑事责任。;中华人民共和国数据安全法第七条国家保护个人、组织与数据有关的权益，鼓励数据 依法合理有效利用，保障数据依法有序自由流动，促进以数据为关键要素的数字经济 发展。;中华人民共和国数据安全法第四十六条违反本法第三十一条规定，向境外提供重要数 据的，由有关主管部门责令改正，给予警告，可以并处十万元以上一百万元以下罚 款，对直接负责的主管人员和其他直接责任人员可以处一万元以上十万元以下罚款； 情节严重的，处一百万元以上一千万元以下罚款，并可以责令暂停相关业务、停业整 顿、吊销相关业务许可证或者吊销营业执照，对直接负责的主管人员和其他直接责任 人员处十万元以上一百万元以下罚款。;中华人民共和国数据安全法第三十八条国家机关为履行法定职责的需要收集、使用数 据，应当在其履行法定职责的范围内依照法律、行政法规规定的条件和程序进行；对 在履行职责中知悉的个人隐私、个人信息、商业秘密、保密商务信息等数据应当依法 予以保密，不得泄露或者非法向他人提供。;中华人民共和国数据安全法第十四条国家实施大数据战略，推进数据基础设施建设， 鼓励和支持数据在各行业、各领域的创新应用。省级以上人民政府应当将数字经济发 展纳入本级国民经济和社会发展规划，并根据需要制定数字经济发展规划。;中华人民共和国数据安全法第四十